In [1]:
knitr::opts_chunk$set(echo = TRUE)
library(caret)
library(ranger)

Loading required package: lattice
Loading required package: ggplot2


In [2]:
df <- read.csv("hmeq.csv")
QuickClean = function(x,i)
{
  if(i==0)
  {
    x = x[complete.cases(x), ]
    return(x)
  }
  if(i==1)
  {
    checkcol = function(y)
    {
      if(is.numeric(y) == TRUE)
      {
        y[is.na(y)] = mean(y, na.rm = TRUE)
      }
      else
      {
        levels=unique(y)
        y[is.na(y)]=levels[which.max(tabulate(match(y, x = levels)))]
      }
      return(y)
    }
    x = lapply(x,checkcol)
    return(data.frame(x))
  }
}
df = QuickClean(df, 1)
df$BAD = as.factor(df$BAD)
names(df)[1] = 'target'

Warning message in file(file, "rt"):
"cannot open file 'hmeq.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [3]:
splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
train <- df[ splitIndex,]
test <- df[-splitIndex,]

model = ranger(target ~., data = train)
pred = predict(model, data = test)$predictions
cm = confusionMatrix(pred, test$target, positive="1")

prop.table(table(df$target))

ERROR: Error in df$target: object of type 'closure' is not subsettable


In [4]:
train1 <- train[train$target == "1",]
train0 <- train[train$target == "0",]

n0 = nrow(train0)
n1 = nrow(train1)
train00 <- train0[sample(1:n0, n1),]

train_under = rbind(train00, train1)
model_under = ranger(target~., data = train_under)
pred_under = predict(model_under, data = test)$predictions
cm_under = confusionMatrix(pred_under, test$target, positive="1")

cm_under
prop.table(table(train_under$target))

ERROR: Error in train$target: object of type 'closure' is not subsettable


In [5]:
train1 <- train[train$target == "1",]
train0 <- train[train$target == "0",]

n0 = nrow(train0)
n1 = nrow(train1)
train11 <- train1[sample(1:n1, n0, replace = TRUE),]

train_over = rbind(train11, train0)
model_over = ranger(target~., data = train_over)
pred_over = predict(model_over, data = test)$predictions
cm_over = confusionMatrix(pred_over, test$target, positive="1")

cm_over
prop.table(table(train_over$target))

ERROR: Error in train$target: object of type 'closure' is not subsettable


In [6]:
Quick_bal = function(df, m){
  if(m == 1) {
    splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
    train <- df[ splitIndex,]
    test <- df[-splitIndex,]
    
    train1 <- train[train$target == "1",]
    train0 <- train[train$target == "0",]

    n0 = nrow(train0)
    n1 = nrow(train1)
    bal_under <- train0[sample(1:n0, n1),]
  }
  else if(m == 2) {
    splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
    train <- df[ splitIndex,]
    test <- df[-splitIndex,]
    
    train1 <- train[train$target == "1",]
    train0 <- train[train$target == "0",]

    n0 = nrow(train0)
    n1 = nrow(train1)
    bal_over <- train1[sample(1:n1, n0, replace = TRUE),]
  }
  else print("Please choose 1 for undersampling or 2 for oversampling")
}

In [7]:
Quick_model_bal = function(df){
  splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
  train <- df[ splitIndex,]
  test <- df[-splitIndex,]
  
  train1 <- train[train$target == "1",]
  train0 <- train[train$target == "0",]
  
  nu0 = nrow(train0)
  nu1 = nrow(train1)
  train00 <- train0[sample(1:nu0, nu1),]
  train_under = rbind(train00, train1)
  model_under = ranger(target~., data = train_under)
  pred_under = predict(model_under, data = test)$predictions
  cm_under = confusionMatrix(pred_under, test$target, positive="1")
  
  no0 = nrow(train0)
  no1 = nrow(train1)
  train11 <- train1[sample(1:no1, no0, replace = TRUE),]
  train_over = rbind(train11, train0)
  model_over = ranger(target~., data = train_over)
  pred_over = predict(model_over, data = test)$predictions
  cm_over = confusionMatrix(pred_over, test$target, positive="1")
  
  print("First cm is for undersampling, second is for oversampling")
  print(cm_under)
  print(cm_over)
}